In [5]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from keras.utils import pad_sequences
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
import io
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm , trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from langchain.text_splitter import TokenTextSplitter
import nltk
# nltk.download('punkt')  # Download the punkt tokenizer for sentence tokenization
from nltk.tokenize import sent_tokenize
import math
from concurrent.futures import ThreadPoolExecutor

In [2]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

GEC_tokenizer = AutoTokenizer.from_pretrained("grammarly/coedit-xl")
GEC_model = T5ForConditionalGeneration.from_pretrained("grammarly/coedit-xl")

# Load the saved state dictionary into the modexl
model_file = '/home/ubuntu/ritesh_manchikanti/cat_poc_model_2/bert-based-uncased-GED.pth'
GED_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
GED_model.load_state_dict(torch.load(model_file))
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(device)
GED_model.to(device)
GED_model.eval()
GED_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case = True)
# # Calculate the number of parameters in the model
# num_params = sum(p.numel() for p in GED_model.parameters())

# # Convert to GB
# size_gb = num_params * 4 / (1024 ** 3)
# print("Size of the Model in GB:", size_gb)

cuda:2


In [15]:


# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        resource_manager = PDFResourceManager()
        fake_file_handle = io.StringIO()
        converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
        page_interpreter = PDFPageInterpreter(resource_manager, converter)
        
        for page in PDFPage.get_pages(file, check_extractable=True):
            page_interpreter.process_page(page)
            text += fake_file_handle.getvalue()
        
        converter.close()
        fake_file_handle.close()
    
    return text
# Function to divide text into chunks of 512 tokens
def divide_text_into_chunks(text, max_chunk_size=512):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        # print(sentence)
        # print("##########")
        tokenized_sentence = sentence.split()
        if len(current_chunk) + len(tokenized_sentence) <= max_chunk_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

def divide_text_into_batches(text, batch_size=5):
    sentences = nltk.sent_tokenize(text)
    num_sentences = len(sentences)
    num_batches = math.ceil(num_sentences / batch_size)

    batches = []
    start_idx = 0
    for batch_idx in range(num_batches):
        end_idx = min(start_idx + batch_size, num_sentences)
        batch_sentences = sentences[start_idx:end_idx]
        batches.append(" ".join(batch_sentences))
        start_idx = end_idx

    return batches

# text_splitter_1 = TokenTextSplitter(chunk_size=100,chunk_overlap = 2)
# Example usage
pdf_path = './15031-4983-FullBook.pdf'
text = extract_text_from_pdf(pdf_path)
chunks = divide_text_into_chunks(text)
batches = divide_text_into_batches(text, 10)
print(len(chunks))
print(len(batches))



def check_grammar(sentences_input, model, tokenizer):
    sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences_input]
    labels = [0]  # Dummy labels for prediction, you can ignore this since we only predict the first sentence.

    # Tokenize and pad the sentences
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    MAX_LEN = 128
    input_ids = pad_sequences(
        [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], 
        maxlen=MAX_LEN, dtype="long", truncating="post", padding="post"
    )
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)
    # Convert inputs to tensors
    prediction_inputs = torch.tensor(input_ids)
    prediction_masks = torch.tensor(attention_masks)
    prediction_labels = torch.tensor(labels)
    # print("prediction_labels",prediction_labels)
    # Set device to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        logits = model(prediction_inputs.to(device), token_type_ids=None, attention_mask=prediction_masks.to(device))
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    # Store predictions and true labels
    predictions = np.argmax(logits, axis=1).flatten()
    zero_indexes = np.where(predictions == 0)[0]
    # print(zero_indexes)
    # print(type(zero_indexes))
    incorrect_senteces = np.array(sentences_input)[zero_indexes]
    return incorrect_senteces



def process_batch(batch, model, tokenizer):
    # print("new_batch")
    sentences = sent_tokenize(batch)
    incorrect_sentences = check_grammar(sentences, GED_model, GED_tokenizer)
    corrected_sentences = correct_grammer(incorrect_sentences, GEC_model, GEC_tokenizer)
    return corrected_sentences
def correct_grammer(input_texts,model,tokenizer):
    
    # input_text = 'Fix grammatical errors in this sentence: When I grow up, I start to understand what he said is quite right.'
    input_ids = [tokenizer(text, return_tensors="pt").input_ids[0] for text in input_texts]
    if len(input_ids) == 0:
        return []
    input_ids = pad_sequences(input_ids, maxlen=256, dtype="long", truncating="post", padding="post", value=tokenizer.pad_token_id)
    input_ids = torch.tensor(input_ids)
    outputs = model.generate(input_ids, max_length=256)
    edited_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    # print(edited_texts)
    return edited_texts
executor = ThreadPoolExecutor(max_workers=4)
all_corrected_sentences = []
for batch_num, corrected_sentences_batch in enumerate(executor.map(lambda x: process_batch(x, GED_model, GED_tokenizer), batches)):
    all_corrected_sentences.extend(corrected_sentences_batch)
    print("Processed Batch",batch_num)
    # print(f"Processed Batch {batch_num + 1}/{len(batches)}")

    
# for idx, chunk in enumerate(batches):
#     print("batch Number :",idx)
#     sentences = sent_tokenize(chunk)
#     chunk_label = check_grammar(sentences, GED_model, GED_tokenizer)
#     break
        
        

76437
33193
Processed Batch 0
Processed Batch 1
Processed Batch 2
Processed Batch 3
Processed Batch 4
Processed Batch 5
Processed Batch 6


RuntimeError: cannot reshape tensor of 0 elements into shape [0, -1, 32, 64] because the unspecified dimension size -1 can be any value and is ambiguous

In [ ]:

def get_first_sentence_prediction(model,tokenizer, sentences):
    sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
    labels = [0]  # Dummy labels for prediction, you can ignore this since we only predict the first sentence.

    # Tokenize and pad the sentences
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    MAX_LEN = 128
    input_ids = pad_sequences(
        [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], 
        maxlen=MAX_LEN, dtype="long", truncating="post", padding="post"
    )
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)
    # Convert inputs to tensors
    prediction_inputs = torch.tensor(input_ids)
    prediction_masks = torch.tensor(attention_masks)
    prediction_labels = torch.tensor(labels)
    # print("prediction_labels",prediction_labels)
    # Set device to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        logits = model(prediction_inputs.to(device), token_type_ids=None, attention_mask=prediction_masks.to(device))
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    # Store predictions and true labels
    predictions = np.argmax(logits, axis=1).flatten()
    return predictions


In [ ]:
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp
from multiprocessing import Manager

def process_chunk(chunk):
    sentences = sent_tokenize(chunk)
    chunk_label = check_grammar(sentences, model, tokenizer)
    return chunk_label

def init_process(rank, size):
    torch.cuda.set_device(rank)

if __name__ == "__main__":
    num_gpus = 4

    # Spawn a process for each GPU
    mp.spawn(init_process, args=(num_gpus,), nprocs=num_gpus)

    # Initialize the model and tokenizer on GPU 0
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    model.load_state_dict(torch.load(model_file))
    model.to(device)
    model = DDP(model, device_ids=[0, 1, 2, 3], output_device=0)  # Use all 4 GPUs for DDP

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

    # ... Rest of your code ...

    # Divide chunks into equal parts for each GPU
    chunks_per_gpu = len(chunks) // num_gpus
    chunks_divided = [chunks[i:i + chunks_per_gpu] for i in range(0, len(chunks), chunks_per_gpu)]

    # Create a Manager to share the results across processes
    manager = Manager()
    grammatical_error_count = manager.list()

    with mp.Pool(processes=num_gpus) as pool:
        pool.map_async(process_chunk, chunks_divided, callback=grammatical_error_count.extend)

    # Wait for all processes to finish
    pool.close()
    pool.join()

    # Calculate the final grammatical error count
    total_error_count = grammatical_error_count.count(0)
    print("Total Grammatical Error Count:", total_error_count)


In [ ]:
def divide_text_into_chunks(text, max_chunk_size=512):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        # print(sentence)
        # print("##########")
        tokenized_sentence = sentence.split()
        if len(current_chunk) + len(tokenized_sentence) <= max_chunk_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

pdf_path = './15031-4983-FullBook.pdf'
text = extract_text_from_pdf(pdf_path)
chunks = divide_text_into_chunks(text[1:1000])
print(len(chunks))
for idx, chunk in enumerate(chunks[:10]):
    sentences = sent_tokenize(chunk)
    label_mapping = {0: "incorrect", 1: "correct"}
    for sentence in sentences:
        
    
    

In [ ]:
# import torch
# from transformers import BertForSequenceClassification, BertTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm , trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
# Tokenize Inputs

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case = True)
# Replace 'path_to_model_file.pth' with the path to the saved .pth model file.
model_file = '/home/ubuntu/ritesh_manchikanti/cat_poc_model_2/bert-based-uncased-GED.pth'

# Create an instance of the model's class
# model = BertForSequenceClassification()

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                      num_labels=2)
# Load the saved state dictionary into the model
model.load_state_dict(torch.load(model_file))

# Optional: If you want to use the model for inference, move it to the appropriate device (GPU or CPU).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# run on a sample text

model.eval()

# Create sentence) and label lists
sentences = ["I have  dog as pet."]
# We need to add special tokens at the beginning and end of each sentence
# for BERT to work properly
print("sentences")
print(sentences)
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels =[0]

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print("tokenized_texts")
print(tokenized_texts)
# Padding Sentences
# Set the maximum sequence length. The longest sequence in our training set
# is 47, but we'll leave room on the end anyway.
# In the original paper, the authors used a length of 512.
MAX_LEN = 128
predictions = []
# Pad our input tokens
input_ids = pad_sequences(
    [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], 
    maxlen=MAX_LEN, dtype="long", truncating="post", padding="post"
    )
# Index Numbers and Padding
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# pad sentences
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype ="long", truncating="post",padding ="post")
# Attention masks
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i > 0) for i in seq]
  attention_masks.append(seq_mask)
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
# Put model in evaluation mode
model.eval()
with torch.no_grad():
  # Forward pass, calculate logit predictions
  logits = model(prediction_inputs.to(device), token_type_ids=None, attention_mask=prediction_masks.to(device))
# Move logits and labels to CPU
logits = logits.detach().cpu().numpy()
# label_ids = b_labels.to("cpu").numpy()
# Store predictions and true labels
predictions.append(logits)
# true_labels.append(label_ids)
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
# flat_true_labels = [item for sublist in true_labels for item in sublist]
print(flat_predictions[0])

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Image

def image_to_pdf(image_path, output_pdf_path):
    doc = SimpleDocTemplate(output_pdf_path, pagesize=letter)
    story = []
    image = Image(image_path)
    image.drawHeight = 700  # Set the height of the image in the PDF (adjust as needed)
    image.drawWidth = 500   # Set the width of the image in the PDF (adjust as needed)
    story.append(image)
    doc.build(story)

# Example usage:
image_path = "./table_ss.png"
output_pdf_path = "./table_ss.pdf"
image_to_pdf(image_path, output_pdf_path)


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Image
from PIL import Image as PILImage

def image_to_pdf(image_path, output_pdf_path):
    # Open the image and get its dimensions
    img = PILImage.open(image_path)
    img_width, img_height = img.size

    # Get the page size (letter size in this case)
    page_width, page_height = letter

    # Calculate the scaling factor to fit the image within the page
    scale_factor = min(page_width / img_width, page_height / img_height)

    # Calculate the scaled image dimensions
    new_width = int(img_width * scale_factor)
    new_height = int(img_height * scale_factor)

    # Create a PDF document
    doc = SimpleDocTemplate(output_pdf_path, pagesize=letter)
    story = []

    # Resize the image to fit the page
    img_resized = img.resize((new_width, new_height), PILImage.ANTIALIAS)

    # Add the resized image to the story
    image = Image(image_resized, width=new_width, height=new_height)
    story.append(image)

    # Build the PDF document
    doc.build(story)

# Example usage:
image_path = "path/to/your/image.jpg"
output_pdf_path = "path/to/your/output.pdf"
image_to_pdf(image_path, output_pdf_path)
